In [1]:
import os

from OCC.Core.IFSelect import IFSelect_RetDone
from OCC.Core.STEPCAFControl import STEPCAFControl_Reader
from OCC.Core.TCollection import TCollection_ExtendedString
from OCC.Core.TDocStd import TDocStd_Document
from OCC.Core.VrmlAPI import VrmlAPI_Writer
from OCC.Core.XCAFDoc import XCAFDoc_ShapeTool, XCAFDoc_DocumentTool
from OCC.Core.BRep import BRep_Tool
from OCC.Core.TDF import TDF_LabelSequence, TDF_Label
from OCC.Extend.TopologyUtils import TopologyExplorer
from OCC.Extend.TopologyUtils import is_edge, is_wire, discretize_edge, discretize_wire
from OCC.Extend.DataExchange import read_step_file_with_names_colors, read_step_file
from OCC.Display.WebGl.x3dom_renderer import X3DExporter, X3DomRenderer
from OCC.Core.TopAbs import topabs
from OCC.Core.BRepMesh import BRepMesh_IncrementalMesh
import xml.etree.ElementTree as ET
from IPython.display import HTML
import x3d.x3d as X3D
import re

x3d.py package loaded, have fun with X3D Graphics!


In [2]:
#stp_filename = 'VentilatorAP203' #'as1_pe_203'
stp_filename = 'as1_pe_203'
stp_path = os.path.join('..','assets', 'step', stp_filename + '.stp')

# load the STEP file
doc = TDocStd_Document(TCollection_ExtendedString("pythonocc-doc"))

step_reader = STEPCAFControl_Reader()
step_reader.SetColorMode(True)
step_reader.SetLayerMode(True)
step_reader.SetNameMode(True)
step_reader.SetMatMode(True)
step_reader.SetGDTMode(True)

status = step_reader.ReadFile(stp_path)
if status == IFSelect_RetDone:
    step_reader.Transfer(doc)
else:
	raise IOError("STEP file could not be read.")

In [3]:
#tdfdata=doc.GetData()
#tdfroot = tdfdata.Root()
# poly=BRep_Tool(tdfroot, )
shapes = XCAFDoc_DocumentTool().ShapeTool(doc.Main())
freelabels = TDF_LabelSequence()
shapes.GetFreeShapes(freelabels)
#rootShape = shapes.GetShape(tdfroot)
freeShape = shapes.GetShape(freelabels.Value(1))
BRepMesh_IncrementalMesh(freeShape, 0.01, True)
# export the document
vrml_exporter = VrmlAPI_Writer()
#vrml_exporter.Write(freeShape, 'tmp.wrl')
vrml_exporter.WriteDoc(doc, stp_filename + ".wrl", 1.0)

True

In [4]:
import subprocess as proc
proc.run(['chmod', 'a+x', '../bin/tovrmlx3d'])
#out = proc.run(['../bin/tovrmlx3d', '--encoding', 'xml', stp_filename + '.wrl'], stdout=open(stp_filename + '.x3d','wb'))
out = proc.run(['../bin/tovrmlx3d', '--encoding', 'xml', stp_filename + '.wrl'], stdout=proc.PIPE)

In [5]:
x3domHTML = '''<script type='text/javascript' src='https://www.x3dom.org/download/dev/x3dom-full.debug.js'> </script> 
<link rel='stylesheet' type='text/css' href='https://www.x3dom.org/download/dev/x3dom.css'></link>'''
#x3domXML = ET.tostring(ET.XML(out.stdout), encoding="unicode", short_empty_elements=False)
x3dXML = ET.XML(out.stdout)
vp=ET.fromstring(X3D.Viewpoint(position=(-1269, 0, 5531)).XML())
sceneElement = list(x3dXML.iter('Scene'))[0]
sceneElement.append(vp)
x3domXML = ET.tostring(x3dXML, encoding="unicode", short_empty_elements=False)

In [6]:
HTML(x3domHTML+x3domXML)

In [7]:
#help(vrml_exporter.Write)

In [8]:
#vrml_exporter.SetRepresentation(1)

In [15]:
#Add wireframe by using with_names_colors to get all subshapes, and with X3DExporter to get the full polygons
stpShapes = read_step_file_with_names_colors(stp_path)
x3domRenderer = X3DomRenderer(path="./")
for shape in stpShapes:
    if (is_edge(shape) | is_wire(shape)):
        continue
    color = stpShapes[shape][1]
    colorRGB=(color.Red(), color.Green(), color.Blue())
    x3dexp = X3DExporter(shape, None, None, True, colorRGB, (0,0,0), 0, 0, (0,0,0), 2, 1)
    x3dexp.compute()
    print(len(x3dexp._triangle_sets), len(x3dexp._line_sets))
    xml=ET.XML(x3dexp.to_x3dfile_string(0))
    shp_vertexCounts=[]
    shp_points=[]
    for lineset in xml.iter('LineSet'):
        shp_vertexCounts.append(lineset.get('vertexCount'))
        shp_points.append(list(lineset.iter('Coordinate'))[0].get('point'))
    #print (shp_vertexCounts, shp_points)
    #reuse first lineset to hold all ines
    lset=list(xml.iter('LineSet'))[0]
    lset.set('vertexCount',",".join(shp_vertexCounts))
    coords=list(lset.iter('Coordinate'))[0]
    coords.set('point',",".join(shp_points))
    #remove all other lineset shapes
    group=next(xml.iter('Group'))
    transforms=list(group.iter('Transform'))
    #skip first
    for transform in transforms[1:]:
        group.remove(transform)
    sceneElement.append(transforms[0])
    #print(ET.dump(group))
    #xml.write("M_"+hash+'.x3d')
    #print(x3dexp.to_x3dfile_string(0))
    #color = stpShapes[shape][1]
    #colorRGB=(color.Red(), color.Green(), color.Blue())
    #x3domRenderer.DisplayShape(shape=shape, color=colorRGB, export_edges=True)


Number of shapes at root : 1

Name : AS1_PE_ASM
Name : PLATE
Name : SOLID
    instance color Name & RGB:  <class 'Quantity_Color'> 229 0.0 1.0 0.0
Name : COMPOUND
    shape color Name & RGB:  <class 'Quantity_Color'> 510 1.0 1.0 0.0
    shape color Name & RGB:  <class 'Quantity_Color'> 510 1.0 1.0 0.0
    shape color Name & RGB:  <class 'Quantity_Color'> 510 1.0 1.0 0.0
    shape color Name & RGB:  <class 'Quantity_Color'> 510 1.0 1.0 0.0
    shape color Name & RGB:  <class 'Quantity_Color'> 510 1.0 1.0 0.0
    shape color Name & RGB:  <class 'Quantity_Color'> 510 1.0 1.0 0.0
    shape color Name & RGB:  <class 'Quantity_Color'> 510 1.0 1.0 0.0
Name : L_BRACKET_ASSEMBLY_ASM
Name : L-BRACKET
Name : SOLID
    instance color Name & RGB:  <class 'Quantity_Color'> 510 1.0 1.0 0.0
Name : COMPOUND
    shape color Name & RGB:  <class 'Quantity_Color'> 510 1.0 1.0 0.0
    shape color Name & RGB:  <class 'Quantity_Color'> 510 1.0 1.0 0.0
    shape color Name & RGB:  <class 'Quantity_Color'> 510

In [16]:
#wireX3D=ET.XML(out.stdout)
#wireGroup=list(wireX3D.iter('Group'))[0]
#sceneElement.append(wireGroup)

In [17]:
x3domWireXML = ET.tostring(x3dXML, encoding="unicode", short_empty_elements=False)

In [18]:
HTML(x3domHTML+x3domWireXML)